# Step 0: Initial set up

Note this is for the type 2 simulation - both galaxies have all 3 components (disk, bulge and halo). See 'main_analysis' for extra comments on and descriptions of the functions below

In [1]:
import os
import sys
import pandas as pd
import numpy as np

#Find the current directory
current_dir = os.getcwd()

#Load extra_functions.py in the current_dir
sys.path.append(current_dir)

import extra_functions as ef

#Write the filepath to the jobs_back folder
jobs_back = os.path.join(current_dir, 'jobs_back')

#Create a list of all the folders in the jobs_back folder
folders = os.listdir(jobs_back)

print(folders)
#List the particle types to gather data for 

particle_types = ['0', '1', '2', '3', '4','5']

#Note if you change these you will also have to change the timesteps (in the top spot) in extra_functions.py
start_timestep = 4
end_timestep = 6

#Create a dataframe for the paramaters
paramaters = pd.DataFrame(columns = ['job_number', ' particles type'])
paramaters_base = pd.DataFrame(columns = ['job_number', ' particles type'])

#Turn off warnings in the terminal

import warnings
warnings.filterwarnings('ignore')


['1069', '2085', '1056', '2071', '2049', '2076', '1051', '2082', '1058', '1093', '2040', '1067', 'Control2', '2078', '1060', '2047', '1094', '1034', '1206', '2013', '2014', '1201', '1033', '2022', '1005', '1208', '1002', '1230', '2025', '2079', 'Control3', '2046', '1095', '1061', '1059', '1066', '1092', '2041', '2048', '1050', '2083', '2077', '1068', '2070', '2084', '1057', '2024', '1231', '1003', '1004', '2023', '1209', '1032', '1200', '2015', '2012', '1207', '1035', '1178', '1147', '1140', '1149', '1182', '1176', '.DS_Store', '1171', '1185', '1125', '2102', '2105', '1122', '1114', '1113', '1184', '1170', '1148', '1177', '1183', '1141', '1179', '1146', '1112', '1115', '1123', '2104', '2103', '1124', '1139', '1106', '2119', '1101', '1108', '1137', '2110', '2117', '1130', '1190', '1164', '1163', '1197', '1155', '1152', '1199', '1131', '2116', '1109', '2111', '1136', '2118', '1100', '1138', '1107', '2120', '1153', '1198', '1154', '1196', '1162', '1165', '1191', '1028', '2030', '1017', '1

In [ ]:
#pull in the folder 1000 job into data and clean it
df = ef.adjust_data(ef.read_data(1036))
df = ef.remove_first_timestep(ef.add_vector_length(df))


# Step 1: Load the base (same as 'control' in type 1)
Note: The base are 3 simulations I ran to compare to the interractions - everything but the velocity and initial position is the same (velocity is 000 and the initial position is far away from galaxy 1 to avoid interractions here)

In [2]:
#Write a loop that will loop over the folders and build up a df_base from the average of 3 simulations

from collections import defaultdict


#These are the functions to analyse
functions = [ef.calculate_eccentricity, ef.calculate_relative_density_table, ef.calculate_fwhm_2D, ef.calculate_fwhm_3D, ef.calculate_fwhm_method_1, ef.calculate_half_mass_radius, ef.calculate_scale_length, ef.calculate_surface_brightness, ef.calculate_metrics]
#functions = [ef.calculate_average_relative_density]

def get_base(start_timestep, end_timestep, particle_types, folders, functions):
    bas_raw = pd.DataFrame(columns = ['job_number', ' particles type','timestep_val'])
    bas_summary = pd.DataFrame(columns = ['job_number', ' particles type','timestep_val'])
    for folder in folders:
        #If the word 'Base' is in the name of the folder, then we want to analyse it
        if 'Base' in folder:
            # Need to clean the data!!!
            df = ef.adjust_data(ef.read_data(folder))
            df = ef.remove_first_timestep(ef.add_vector_length(df))

            #loop over the functions to pull the data
            for func in functions:
                base_summary_temp, base_raw_temp = ef.analyse_function_range(func, df, particle_types, start_timestep, end_timestep, folder)
                #drop 'folder' column
                base_summary_temp = base_summary_temp.drop(['folder'], axis = 1)
                base_raw_temp = base_raw_temp.drop(['folder'], axis = 1)
                # Append the new data to bas_summary an#d bas_raw
                bas_summary = bas_summary.merge(base_summary_temp, on=[' particles type', 'timestep_val'], how='outer')
                bas_raw = bas_raw.merge(base_raw_temp, on=[' particles type', 'timestep_val'], how='outer')  
        else:
            continue
    # Add in the job_number column as 'Base'
    bas_raw['job_number'] = 'Base'
    bas_summary['job_number'] = 'Base'

    # Calculate the averages for columns with the same suffix
    grouped_cols = defaultdict(list)
    for col in bas_summary.columns:
        if 'Base' in col:
            attribute = col.split('Base')[1][1:] # Split on 'Base' and remove the first character (number) 
            grouped_cols[attribute].append(col)
            
    for attribute, cols in grouped_cols.items():
        if len(cols) > 1: # Only compute mean if there is more than one column to average
            bas_summary['Base'+attribute] = bas_summary[cols].mean(axis=1)
            bas_raw['Base'+attribute] = bas_raw[cols].mean(axis=1)
            bas_summary.drop(cols, axis=1, inplace=True)
            bas_raw.drop(cols, axis=1, inplace=True)

    return bas_summary, bas_raw

#Run the function to add new data from get_base to the paramaters_base df - dont override the paramaters_base df
bas_summary, bas_raw = get_base(start_timestep, end_timestep, particle_types, folders, functions)

#Save the two dataframes to excel files which can be referenced later on - make the excel file have a timestamp and place in a folder 'data'
import datetime
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d_%H-%M-%S")
#create the data folder if it isnt there
if not os.path.exists('data'):
    os.makedirs('data')
    
bas_summary.to_excel('data/base_summary.xlsx')
bas_raw.to_excel('data/base_raw.xlsx')




File exists
Error occurred for particle type: 1, folder: Base1, function: calculate_fwhm_2D
Error message: Optimal parameters not found: Number of calls to function has reached maxfev = 800.
Error occurred for particle type: 1, folder: Base1, function: calculate_fwhm_3D
Error message: Optimal parameters not found: Number of calls to function has reached maxfev = 800.
/Users/a/Downloads/ast80011-main/gi/batchrunner/jobs_back/Base2/output/Galaxy Interaction/_output_ascii.csv.gz
Error occurred for particle type: 1, folder: Base2, function: calculate_fwhm_2D
Error message: Optimal parameters not found: Number of calls to function has reached maxfev = 800.
Error occurred for particle type: 1, folder: Base2, function: calculate_fwhm_3D
Error message: Optimal parameters not found: Number of calls to function has reached maxfev = 800.
/Users/a/Downloads/ast80011-main/gi/batchrunner/jobs_back/Base3/output/Galaxy Interaction/_output_ascii.csv.gz
Error occurred for particle type: 1, folder: Base3

# Step 2: Processing the main data

This is the main section which will calculate the whole array here of the data - if you have a lot of simulations it can take a long time so break it down if you need by limiting the 'folders'.

Note not everything will properly analyse - the fwhm for example requires there to be a shape it can estimate the gausian over which means if it is compleatly destroyed or spread out in the event it will not work

In [15]:
import os
import datetime
import traceback
# Write a loop that will loop over the folders and build up df_base from the average of 3 simulations

# These are the functions to analyze
functions = [ef.calculate_eccentricity, ef.calculate_relative_density_table, ef.calculate_fwhm_2D, ef.calculate_fwhm_3D, ef.calculate_fwhm_method_1, ef.calculate_half_mass_radius, ef.calculate_scale_length, ef.calculate_surface_brightness, ef.calculate_metrics]

# Snip the folders variable to 5
#folders = folders[:5]

# Initialize an Excel file to write the data to
import datetime
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d_%H-%M-%S")
# Create the data folder if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

def get_data(start_timestep, end_timestep, particle_types, folders, functions):
    processed_job_numbers = set()
    # read in the excel file 'data_summary' if it exists
    if os.path.exists('data/type2_summary.csv'):
        data_summary = pd.read_csv('data/type2_summary.csv')
        #add the data_summary column 'folder' to the processed_job_numbers set
        processed_job_numbers = set(data_summary['folder'])
        #else continue as normal
    else:

        pass
    error_folders = []
    for folder in folders:
        try:
            # If the folder name starts with a 1, we want to analyze it
            if folder[0] == '2':
                job_number = folder.split('_')[0]  # Extract job number from folder name

                # Check if job number already exists in the processed job numbers set
                if job_number in processed_job_numbers:
                    continue  # Skip this folder

                # Need to clean the data!!!
                df = ef.adjust_data(ef.read_data(folder))
                df = ef.remove_first_timestep(ef.add_vector_length(df))

                # Initialize the dataframes
                data_raw = pd.DataFrame(columns=[' particles type', 'timestep_val'])
                data_summary = pd.DataFrame(columns=[' particles type', 'timestep_val'])

                # Loop over the functions to pull the data
                for func in functions:
                    data_summary_temp, data_raw_temp = ef.analyse_function_range(func, df, particle_types, start_timestep, end_timestep, folder)
                    # Append the new data to data_summary and data_raw
                    data_summary = data_summary.merge(data_summary_temp, on=[' particles type', 'timestep_val'], how='outer')
                    data_raw = data_raw.merge(data_raw_temp, on=[' particles type', 'timestep_val'], how='outer')

                #Remove all folder_x and all folder_y columns
                data_summary = data_summary.drop(columns=['folder_y'])
                data_summary = data_summary.drop(columns=['folder_x'])
                data_raw = data_raw.drop(columns=['folder_y'])
                data_raw = data_raw.drop(columns=['folder_x'])

                #Remove the job number from the column headers
                data_summary.columns = [col.replace(job_number+'_', '') for col in data_summary.columns]

                # Write data_summary to the CSV file
                with open(f'data/type2_summary.csv', 'a') as f_summary:
                    data_summary.to_csv(f_summary, header=f_summary.tell() == 0, index=False)

                # Write data_raw to the CSV file
                with open(f'data/type2_raw.csv', 'a') as f_raw:
                    data_raw.to_csv(f_raw, header=f_raw.tell() == 0, index=False)

                # Update the processed job numbers set
                processed_job_numbers.add(job_number)
        except Exception as e:
            error_folders.append(folder)
            print(f"Error occurred in folder: {folder}")
            print(f"Error message: {str(e)}")
            print("Traceback:")
            traceback.print_exc()  # This will print the traceback

    # Print the error folders
    print("Folders with errors:")
    print(error_folders)

# Create the data folder if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

# Get the current timestamp
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d_%H-%M-%S")

# Run the function to add new data from get_data to the CSV files
get_data(start_timestep, end_timestep, particle_types, folders, functions)


/Users/a/Downloads/ast80011-main/gi/batchrunner/jobs_back/2085/output/Galaxy Interaction/_output_ascii.csv.gz
Error occurred for particle type: 5, folder: 2085, function: calculate_fwhm_2D
Error message: Optimal parameters not found: Number of calls to function has reached maxfev = 800.
Error occurred for particle type: 5, folder: 2085, function: calculate_fwhm_3D
Error message: Optimal parameters not found: Number of calls to function has reached maxfev = 800.
/Users/a/Downloads/ast80011-main/gi/batchrunner/jobs_back/2071/output/Galaxy Interaction/_output_ascii.csv.gz
Error occurred for particle type: 5, folder: 2071, function: calculate_fwhm_2D
Error message: Optimal parameters not found: Number of calls to function has reached maxfev = 800.
Error occurred for particle type: 5, folder: 2071, function: calculate_fwhm_3D
Error message: Optimal parameters not found: Number of calls to function has reached maxfev = 800.
/Users/a/Downloads/ast80011-main/gi/batchrunner/jobs_back/2049/outpu

# Step 3: Normalise the data

Analyse and clean some of this data and compare to the base

In [2]:
#Lets read in the summary file for the base and the data
data_summary = pd.read_csv('data/type2_summary.csv')
bas_summary = pd.read_excel('data/base_summary.xlsx')
#Print the column headers of each
#print(data_summary.columns)
#print(bas_summary.columns)

In [3]:

#Cleaning the columns - note this will vary and have to be varied depending on the data set

#Remove 1069 from column headers of data_summary
data_summary.columns = [col.replace('2085', '') for col in data_summary.columns]
#Remove  'Base' from headers of bas_summary
bas_summary.columns = [col.replace('Base', '') for col in bas_summary.columns]

#Drop 'timestep_val' from both
data_summary = data_summary.drop(columns=['timestep_val'])
bas_summary = bas_summary.drop(columns=['timestep_val'])

#Remove any leading _'s from bas_summary - ONLY LEADING - not all
bas_summary.columns = [col.lstrip('_') for col in bas_summary.columns]

In [4]:
#Check the column headers of each
print(data_summary.columns)
print(bas_summary.columns)

Index([' particles type', 'calculate_eccentricity',
       'calculate_relative_density_table', 'calculate_fwhm_2D',
       'calculate_fwhm_3D', 'calculate_fwhm_method_1',
       'calculate_half_mass_radius', 'calculate_scale_length',
       'calculate_surface_brightness', 'folder', 'position_std_dev',
       'mean_velocity', 'velocity_std_dev'],
      dtype='object')
Index(['Unnamed: 0', 'job_number', ' particles type', 'calculate_eccentricity',
       'calculate_relative_density_table', 'calculate_fwhm_2D',
       'calculate_fwhm_3D', 'calculate_fwhm_method_1',
       'calculate_half_mass_radius', 'calculate_scale_length',
       'calculate_surface_brightness', 'position_std_dev', 'mean_velocity',
       'velocity_std_dev'],
      dtype='object')


In [5]:
# Drop 'Unnamed: 0' and 'job_number' from base dataframe
bas_summary.drop(columns=['Unnamed: 0', 'job_number'], inplace=True)

# Merge the bas_summary onto the data_summary - data_summary has many more rows due to having multiple folder types (different simulations). I want to keep all of those and repeat bas_summary. incolude suffixes _data and _base
merged = data_summary.merge(bas_summary, on=[' particles type'], how='left', suffixes=('_data', '_base'))

# Create new dataframe for normalized data
norm_data = merged.copy()

#bring 'folder' and ' particles type' to the front
cols = list(norm_data.columns)
cols.insert(0, cols.pop(cols.index(' particles type')))
cols.insert(0, cols.pop(cols.index('folder')))
norm_data = norm_data.loc[:, cols]

# Loop through columns in merged dataframe
for col in merged.columns:
    # Check if column ends with '_data'
    if col.endswith('_data'):
        # Find corresponding base column
        base_col = col.replace('_data', '_base')
        # Check if corresponding base column exists
        if base_col in merged.columns:
            # Normalize data by dividing data column by corresponding base column
            norm_data[col.replace('_data', '_norm')] = merged[col] / merged[base_col]

In [6]:
print(norm_data.columns)


Index(['folder', ' particles type', 'calculate_eccentricity_data',
       'calculate_relative_density_table_data', 'calculate_fwhm_2D_data',
       'calculate_fwhm_3D_data', 'calculate_fwhm_method_1_data',
       'calculate_half_mass_radius_data', 'calculate_scale_length_data',
       'calculate_surface_brightness_data', 'position_std_dev_data',
       'mean_velocity_data', 'velocity_std_dev_data',
       'calculate_eccentricity_base', 'calculate_relative_density_table_base',
       'calculate_fwhm_2D_base', 'calculate_fwhm_3D_base',
       'calculate_fwhm_method_1_base', 'calculate_half_mass_radius_base',
       'calculate_scale_length_base', 'calculate_surface_brightness_base',
       'position_std_dev_base', 'mean_velocity_base', 'velocity_std_dev_base',
       'calculate_eccentricity_norm', 'calculate_relative_density_table_norm',
       'calculate_fwhm_2D_norm', 'calculate_fwhm_3D_norm',
       'calculate_fwhm_method_1_norm', 'calculate_half_mass_radius_norm',
       'calculate_sc

In [7]:
#Bring in the rest of the simulation data so we can compare
import os
import json
import pandas as pd

# create an empty dataframe to store the initial_params
initial_params = pd.DataFrame(columns=['folder', 'vx', 'vy', 'vz'])

#filter 'folders' so we only pick up those which start with '2'
folders_run = [f for f in folders if f.startswith('2')]

# loop over each subfolder
for folder in folders_run:
    subfolder = os.path.join(jobs_back, str(folder))
    # read the merge.json file for the velocity_galaxy_2 parameters
    merge_file = os.path.join(subfolder, 'merge.json')
    with open(merge_file, 'r') as f:
        data = json.load(f)
        vx = data['velocity_galaxy_2']['vx']
        vy = data['velocity_galaxy_2']['vy']
        vz = data['velocity_galaxy_2']['vz']
    
    # append the results to the dataframe
    new_row = {'folder': folder, 'vx': vx, 'vy': vy, 'vz': vz}
    initial_params = pd.concat([initial_params, pd.DataFrame([new_row])], ignore_index=True)

# print the results
#print(initial_params)


In [8]:
#change the 'folder' column data type to text for both dataframes
initial_params['folder'] = initial_params['folder'].astype(str)
norm_data['folder'] = norm_data['folder'].astype(str)

#Make a list of the folders in the initial_params dataframe that are not in norm_data
missing_folders = [f for f in initial_params['folder'] if f not in norm_data['folder'].tolist()]

#Merge the initial_params onto the norm_data on 'folder'
norm_data_init = norm_data.merge(initial_params, on=['folder'], how='left')
#print(norm_data_init)

In [9]:
#print(norm_data_init)

Now we get to the real analysis where we can analyse the actual data

In [10]:
#Remove where vx = vy
norm_data_init = norm_data_init[norm_data_init.vx != norm_data_init.vy]

In [11]:
#Filter norm_data_init so we remove any of the first lot of simulations - ie the ones which have length of 3 as opposed to 4 numbers
norm_data_init_cl = norm_data_init[norm_data_init['folder'].str.len() == 4]


In [29]:
# Get unique particle types#
#particle_types = df[' particles type'].unique()

particle_types = [5]

#Call plot_all_heatmaps
ef.plot_all_heatmaps(norm_data_init_cl, particle_types, '_norm','type 2')


# Step 4) looking through the different variables

### A) Eccentricity

In [13]:
from matplotlib.colors import ListedColormap
# Usage:
ef.plot_heatmap_decide(norm_data_init_cl, 'calculate_eccentricity','_data', 5, [0, 0.9, 0.95, 1], 'type 2')


### B) Surface brightness

In [ ]:
#Note I didnt manage to get proper ranges for surface brightness

### C) Half mass radius

In [15]:
ef.plot_heatmap_decide(norm_data_init_cl, 'calculate_half_mass_radius','_norm', 4, [0, 1.5, 5, 10], 'type 2',order = 'middle')

### D) FWHM

In [24]:
#   'calculate_fwhm_2D_norm', 'calculate_fwhm_3D_norm','calculate_fwhm_method_1_norm',
ef.plot_heatmap_decide(norm_data_init_cl, 'calculate_fwhm_method_1','_norm', 4, [0, 1.5, 5, 10], 'type 2',order = 'middle')

In [25]:
ef.plot_heatmap_decide(norm_data_init_cl, 'calculate_fwhm_2D','_norm', 4, [0, 1.5, 5, 10], 'type 2',order = 'middle')

In [26]:
ef.plot_heatmap_decide(norm_data_init_cl, 'calculate_fwhm_3D','_norm', 4, [0, 1.5, 5, 10], 'type 2',order = 'middle')

### E) Scale Length

In [31]:
ef.plot_heatmap_decide(norm_data_init_cl, 'calculate_scale_length','_norm', 4, [0, 1.2, 2, 10], 'type 2',order = 'middle')

### F) Density

In [47]:
ef.plot_heatmap_decide(norm_data_init_cl, 'calculate_relative_density_table','_norm', 4, [0, 0.001, 0.01, 1], 'type 2',order ='correct')

# Step 5) Summarise and combining these into a y/n/m table

In [12]:
#Call the headers and summary (note this can be edited in extra_functions)
df_headers = ef.call_headers_summary()
norm_data_init_cl_final = ef.generate_summary(norm_data_init_cl, df_headers)
columns_to_check = [
    'calculate_relative_density_table_summary',
    'calculate_scale_length_summary',
    'calculate_fwhm_method_1_summary',
    'calculate_fwhm_3D_summary',
    'calculate_fwhm_2D_summary',
    'calculate_half_mass_radius_summary'
]

In [13]:
#Call the functions

norm_data_init_cl_final['summary'] = norm_data_init_cl_final.apply(
    lambda row: ef.summarise_data(row, columns_to_check), axis=1)

norm_data_init_cl_final['summary_broad'] = norm_data_init_cl_final.apply(
    lambda row: ef.summarise_data_broad(row, columns_to_check), axis=1)

norm_data_init_cl_final['summary_y'] = norm_data_init_cl_final.apply(
    lambda row: ef.summarise_data_y(row, columns_to_check), axis=1)

norm_data_init_cl_final['summary_m'] = norm_data_init_cl_final.apply(
    lambda row: ef.summarise_data_m(row, columns_to_check), axis=1)

norm_data_init_cl_final['summary_narrow'] = norm_data_init_cl_final.apply(
    lambda row: ef.summarise_data_narrow(row, columns_to_check), axis=1)

print(norm_data_init_cl_final)


      particles type    vx    vy calculate_relative_density_table_summary  \
0                  0  -300  -200                                      NaN   
1                  1  -300  -200                                      NaN   
2                  2  -300  -200                                      NaN   
3                  3  -300  -200                                        y   
4                  4  -300  -200                                        y   
..               ...   ...   ...                                      ...   
721                1  -700  -800                                      NaN   
722                2  -700  -800                                      NaN   
723                3  -700  -800                                        y   
724                4  -700  -800                                        n   
725                5  -700  -800                                        y   

    calculate_scale_length_summary calculate_fwhm_method_1_summary  \
0    

In [18]:
# First, make DataFrame long format
#Limit to only particles type 4 or 5
particle_type_sel = 5
norm_data_init_cl_final_sum = norm_data_init_cl_final[norm_data_init_cl_final[' particles type'] == particle_type_sel]
df_long = norm_data_init_cl_final_sum[['summary', 'summary_narrow', 'summary_broad']].melt()

# Then pivot it using groupby and unstack
df_pivot = df_long.groupby(['variable', 'value']).size().unstack(level=0)
df_pivot = df_pivot.fillna(0)
df_pivot = df_pivot.astype(int)

print(df_pivot)

variable  summary  summary_broad  summary_narrow
value                                           
m               0             12               6
n             110             87              96
y               0             11               8


In [19]:
def replace_values(df, column_name):
    new_column_name = column_name + '_num'
    df[new_column_name] = df[column_name].replace('y', 0.5)
    df[new_column_name] = df[new_column_name].replace('m', 1.5)
    df[new_column_name] = df[new_column_name].replace('n', 2.5)
    return df

# Call the function for summary, summary_narrow and summary_broad
norm_data_init_cl_final = replace_values(norm_data_init_cl_final, 'summary')
norm_data_init_cl_final = replace_values(norm_data_init_cl_final, 'summary_narrow')
norm_data_init_cl_final = replace_values(norm_data_init_cl_final, 'summary_broad')

# Plot the data using the 'plot_heatmap_decide' function
ef.plot_heatmap_decide(norm_data_init_cl_final, 'summary_num', '', particle_type_sel, [0, 1, 2, 3], 'type 2')
ef.plot_heatmap_decide(norm_data_init_cl_final, 'summary_narrow_num', '', particle_type_sel, [0, 1, 2, 3], 'type 2')
ef.plot_heatmap_decide(norm_data_init_cl_final, 'summary_broad_num', '', particle_type_sel, [0, 1, 2, 3], 'type 2')


# Step 4: A look at the bases


In [2]:
# Cleaning the data

#Read in the base raw data
base_raw = pd.read_excel('data/base_raw.xlsx')

#Remove  'Base' from headers of bas_summary
base_raw.columns = [col.replace('Base', '') for col in base_raw.columns]

#Remove any leading _'s from bas_summary - ONLY LEADING - not all
base_raw.columns = [col.lstrip('_') for col in base_raw.columns]

#Drop the unnamed: 0 column
base_raw = base_raw.drop(columns=['Unnamed: 0'])
#Drop the jobnumber column
base_raw = base_raw.drop(columns=['job_number'])

In [31]:
ef.plot_all_bases(base_raw, 5)

Index([' particles type', 'timestep_val', 'calculate_eccentricity',
       'calculate_relative_density_table', 'calculate_fwhm_2D',
       'calculate_fwhm_3D', 'calculate_fwhm_method_1',
       'calculate_half_mass_radius', 'calculate_scale_length',
       'calculate_surface_brightness', 'position_std_dev', 'mean_velocity',
       'velocity_std_dev'],
      dtype='object')

In [4]:
# Plot a new graph of only particle type 5 for relative density with timescale on the axis
import matplotlib.pyplot as plt
#Remove particle type 4
base_raw_5 = base_raw[base_raw[' particles type'] != 4]

ef.plot_bases(base_raw_5, 'calculate_relative_density_table', 5)

#Plot the graph - dont use ef. functions
#plt.plot(base_raw_5['timestep_val'], base_raw_5['calculate_relative_density_table'])